<a href="https://colab.research.google.com/github/zganjei/Product-Recommender-GenAI/blob/main/genAI_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Product recommender system using OpenAI

This is a prototype for recommending products to a user based on their recent product views. The prototype uses a text embedding model from OpenAI API.


## Setting up the environment

In [5]:
# %%capture
!pip install openai

## Importing the required modules and setting up OpenAI API

In [6]:
import pandas as pd
import numpy as np
import os
from openai import OpenAI
from matplotlib import pyplot as plt
import plotly.express as px

from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import userdata
api_key = userdata.get('openai.api_key')
if api_key:
  print(f"API key: {api_key[:20]}....")
else:
  print("API key not found!")


client = OpenAI(
    api_key= api_key
 )

client


API key: sk-proj-5pfnvLzOd8Qq....


## Import the dataset

In [8]:
data = pd.read_csv("products_dataset.csv")
data

,product_id,title,description
0,P0,Men's 3X Large Carbon Heather Cotton/Polyester...,"This heavyweight, water-repellent hooded sweat..."
1,P1,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...,If you need more length between your existing ...
2,P2,Large Tapestry Bolster Bed,Polyester cover resembling rich Italian tapest...
3,P3,16-Gauge-Sinks Vessel Sink in White with Faucet,It features a rectangle shape. This vessel set...
4,P4,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...,This 9 in. black full grain leather logger boo...
...,...,...,...
1995,P1995,Dotty Black and White Black and White Wallpape...,"With a stylish monochrome look, this dotty wal..."
1996,P1996,Abrielle Brown/Light Gray 8 ft. x 10 ft. Orien...,The Abrielle collection features a stunning as...
1997,P1997,20 in. x 2-1/2 in. x 2-1/2 in. Polyurethane As...,"With Fypon balustrade systems, you can transfo..."
1998,P1998,1 gal. #P120-6 Diva Glam Flat Exterior Paint &...,BEHR PREMIUM PLUS Exterior Paint & Primer is a...


List of last 8 products recently viewed by the user

In [9]:
searched_products_id = [
    'P1938',
    'P1970',
    'P1044',
    'P1838',
    'P1048',
    'P1017',
    'P1310',
    'P1444'
]

### Prepare the dataset

Label the data points that are recently viewed



In [10]:
data['product_status'] = "not_viewed"
data.loc[data.product_id.isin(searched_products_id),'product_status'] = "recently_viewed"
data[data.product_status == "recently_viewed"]

,product_id,title,description,product_status
1017,P1017,1 qt. #660D-7 Blackberry Farm Satin Enamel Int...,Love your space like never before with the hig...,recently_viewed
1044,P1044,1 qt. #M360-4 Marjoram One-Coat Hide Eggshell ...,Introducing the best of BEHR Paint. Featuring ...,recently_viewed
1048,P1048,5 gal. #640C-1 Hosta Flower Extra Durable Sati...,BEHR ULTRA SCUFF DEFENSE Stain-Blocking Paint ...,recently_viewed
1310,P1310,5 gal. #180A-2 Romantic Morn Extra Durable Sem...,BEHR ULTRA SCUFF DEFENSE Stain-Blocking Paint ...,recently_viewed
1444,P1444,5 gal. #PPU12-17 Cameroon Green Extra Durable ...,BEHR ULTRA SCUFF DEFENSE Stain-Blocking Paint ...,recently_viewed
1838,P1838,5 gal. #N340-2 Dune Grass Extra Durable Satin ...,BEHR ULTRA SCUFF DEFENSE Stain-Blocking Paint ...,recently_viewed
1938,P1938,1 gal. #HDC-SP16-10 Japanese Rose Garden Semi-...,Introducing the best of BEHR Paint. Featuring ...,recently_viewed
1970,P1970,8 oz. #510C-3 Rivers Edge Semi-Gloss Enamel St...,Introducing the best of BEHR Paint. Featuring ...,recently_viewed


Combine the product ``` title``` and ```description``` and store it in a column called ```combined``` and use it in the recommender system

In [11]:
data['combined'] = data.title + data.description
data

,product_id,title,description,product_status,combined
0,P0,Men's 3X Large Carbon Heather Cotton/Polyester...,"This heavyweight, water-repellent hooded sweat...",not_viewed,Men's 3X Large Carbon Heather Cotton/Polyester...
1,P1,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...,If you need more length between your existing ...,not_viewed,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...
2,P2,Large Tapestry Bolster Bed,Polyester cover resembling rich Italian tapest...,not_viewed,Large Tapestry Bolster BedPolyester cover rese...
3,P3,16-Gauge-Sinks Vessel Sink in White with Faucet,It features a rectangle shape. This vessel set...,not_viewed,16-Gauge-Sinks Vessel Sink in White with Fauce...
4,P4,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...,This 9 in. black full grain leather logger boo...,not_viewed,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...
...,...,...,...,...,...
1995,P1995,Dotty Black and White Black and White Wallpape...,"With a stylish monochrome look, this dotty wal...",not_viewed,Dotty Black and White Black and White Wallpape...
1996,P1996,Abrielle Brown/Light Gray 8 ft. x 10 ft. Orien...,The Abrielle collection features a stunning as...,not_viewed,Abrielle Brown/Light Gray 8 ft. x 10 ft. Orien...
1997,P1997,20 in. x 2-1/2 in. x 2-1/2 in. Polyurethane As...,"With Fypon balustrade systems, you can transfo...",not_viewed,20 in. x 2-1/2 in. x 2-1/2 in. Polyurethane As...
1998,P1998,1 gal. #P120-6 Diva Glam Flat Exterior Paint &...,BEHR PREMIUM PLUS Exterior Paint & Primer is a...,not_viewed,1 gal. #P120-6 Diva Glam Flat Exterior Paint &...


## Text embedding and visualization

Create the text embedding vectors with dimention 512 from an OpenAI text embedding model to compare products

In [12]:
response = client.embeddings.create(
    input = data.combined.to_list(),
    model = "text-embedding-3-small",
    dimensions = 512
)

vectors = [d.embedding for d in response.data]
data['text_embedding'] = vectors
data

,product_id,title,description,product_status,combined,text_embedding
0,P0,Men's 3X Large Carbon Heather Cotton/Polyester...,"This heavyweight, water-repellent hooded sweat...",not_viewed,Men's 3X Large Carbon Heather Cotton/Polyester...,"[0.03711149841547012, 0.030765360221266747, -0..."
1,P1,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...,If you need more length between your existing ...,not_viewed,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...,"[0.03523961082100868, 0.013278326019644737, 0...."
2,P2,Large Tapestry Bolster Bed,Polyester cover resembling rich Italian tapest...,not_viewed,Large Tapestry Bolster BedPolyester cover rese...,"[0.035860564559698105, -0.05905349925160408, 0..."
3,P3,16-Gauge-Sinks Vessel Sink in White with Faucet,It features a rectangle shape. This vessel set...,not_viewed,16-Gauge-Sinks Vessel Sink in White with Fauce...,"[-0.05834035575389862, -0.007969953119754791, ..."
4,P4,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...,This 9 in. black full grain leather logger boo...,not_viewed,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...,"[0.01998496614396572, 0.05075598508119583, -0...."
...,...,...,...,...,...,...
1995,P1995,Dotty Black and White Black and White Wallpape...,"With a stylish monochrome look, this dotty wal...",not_viewed,Dotty Black and White Black and White Wallpape...,"[0.08823681622743607, -0.05279356613755226, -0..."
1996,P1996,Abrielle Brown/Light Gray 8 ft. x 10 ft. Orien...,The Abrielle collection features a stunning as...,not_viewed,Abrielle Brown/Light Gray 8 ft. x 10 ft. Orien...,"[0.010978340171277523, -0.04043574631214142, 0..."
1997,P1997,20 in. x 2-1/2 in. x 2-1/2 in. Polyurethane As...,"With Fypon balustrade systems, you can transfo...",not_viewed,20 in. x 2-1/2 in. x 2-1/2 in. Polyurethane As...,"[-0.034120045602321625, -0.009548034518957138,..."
1998,P1998,1 gal. #P120-6 Diva Glam Flat Exterior Paint &...,BEHR PREMIUM PLUS Exterior Paint & Primer is a...,not_viewed,1 gal. #P120-6 Diva Glam Flat Exterior Paint &...,"[-0.010861445218324661, -0.014231621287763119,..."


Visualize the text embedding vectors

In [13]:
pca = PCA(2)
vector_2d = pca.fit_transform(data.text_embedding.to_list())
data['pc1'] = vector_2d[:,0]
data['pc2'] = vector_2d[:,1]
data

,product_id,title,description,product_status,combined,text_embedding,pc1,pc2
0,P0,Men's 3X Large Carbon Heather Cotton/Polyester...,"This heavyweight, water-repellent hooded sweat...",not_viewed,Men's 3X Large Carbon Heather Cotton/Polyester...,"[0.03711149841547012, 0.030765360221266747, -0...",-0.012656,0.072019
1,P1,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...,If you need more length between your existing ...,not_viewed,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...,"[0.03523961082100868, 0.013278326019644737, 0....",-0.357223,0.236123
2,P2,Large Tapestry Bolster Bed,Polyester cover resembling rich Italian tapest...,not_viewed,Large Tapestry Bolster BedPolyester cover rese...,"[0.035860564559698105, -0.05905349925160408, 0...",-0.201103,-0.206634
3,P3,16-Gauge-Sinks Vessel Sink in White with Faucet,It features a rectangle shape. This vessel set...,not_viewed,16-Gauge-Sinks Vessel Sink in White with Fauce...,"[-0.05834035575389862, -0.007969953119754791, ...",-0.181791,0.043376
4,P4,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...,This 9 in. black full grain leather logger boo...,not_viewed,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...,"[0.01998496614396572, 0.05075598508119583, -0....",-0.214504,0.141175
...,...,...,...,...,...,...,...,...
1995,P1995,Dotty Black and White Black and White Wallpape...,"With a stylish monochrome look, this dotty wal...",not_viewed,Dotty Black and White Black and White Wallpape...,"[0.08823681622743607, -0.05279356613755226, -0...",-0.042120,-0.196934
1996,P1996,Abrielle Brown/Light Gray 8 ft. x 10 ft. Orien...,The Abrielle collection features a stunning as...,not_viewed,Abrielle Brown/Light Gray 8 ft. x 10 ft. Orien...,"[0.010978340171277523, -0.04043574631214142, 0...",-0.246729,-0.481591
1997,P1997,20 in. x 2-1/2 in. x 2-1/2 in. Polyurethane As...,"With Fypon balustrade systems, you can transfo...",not_viewed,20 in. x 2-1/2 in. x 2-1/2 in. Polyurethane As...,"[-0.034120045602321625, -0.009548034518957138,...",-0.082867,0.103890
1998,P1998,1 gal. #P120-6 Diva Glam Flat Exterior Paint &...,BEHR PREMIUM PLUS Exterior Paint & Primer is a...,not_viewed,1 gal. #P120-6 Diva Glam Flat Exterior Paint &...,"[-0.010861445218324661, -0.014231621287763119,...",0.506964,0.005669


In [14]:
px.scatter(data, x='pc1',y='pc2',color='product_status')

## Find similar products

Use cosine similarity between viewed products and not viewed ones

In [15]:
df_recently_viewed = data[data.product_status == "recently_viewed"]
df_not_viewed = data[data.product_status == "not_viewed"]

vectors_recently_viewed = [np.array(vector) for vector in df_recently_viewed.text_embedding]
vectors_not_viewed = [np.array(vector) for vector in df_not_viewed.text_embedding]

similarity_matrix = cosine_similarity(vectors_recently_viewed, vectors_not_viewed)
top_ids = []
for row in similarity_matrix:
  top_id = np.argmax(row)
  top_ids.append(top_id)

most_similar_product_ids = list(df_not_viewed.iloc[top_ids].product_id)
most_similar_product_ids

['P854', 'P1061', 'P1705', 'P733', 'P1327', 'P1705', 'P1059', 'P314']

## Recommend similar products to the user

In [20]:
data.loc[data.product_id.isin(most_similar_product_ids),"product_status"] = "recommended"
data[data.product_status=="recommended"]

,product_id,title,description,product_status,combined,text_embedding,pc1,pc2
314,P314,8 oz. #230F-7 Florence Brown Semi-Gloss Enamel...,Introducing the best of BEHR Paint. Featuring ...,recommended,8 oz. #230F-7 Florence Brown Semi-Gloss Enamel...,"[-0.0039132810197770596, -0.05801989138126373,...",0.486894,-0.059974
733,P733,5 gal. #N440-1 Streetwise Extra Durable Semi-G...,BEHR ULTRA SCUFF DEFENSE Stain-Blocking Paint ...,recommended,5 gal. #N440-1 Streetwise Extra Durable Semi-G...,"[0.010829787701368332, -0.01894751563668251, 0...",0.468493,-0.009418
854,P854,1 qt. #N460-1 Evening White Satin Enamel Inter...,Love your space like never before with the hig...,recommended,1 qt. #N460-1 Evening White Satin Enamel Inter...,"[0.03936273232102394, -0.017813587561249733, 0...",0.493100,-0.052175
1059,P1059,1 gal. Home Decorators Collection #HDC-SP14-6 ...,Introducing the best of BEHR Paint. Featuring ...,recommended,1 gal. Home Decorators Collection #HDC-SP14-6 ...,"[-0.0021748889703303576, -0.05987035483121872,...",0.448100,-0.054042
1061,P1061,1 gal. #MQ1-28 Orange Flambe One-Coat Hide Egg...,Introducing the best of BEHR Paint. Featuring ...,recommended,1 gal. #MQ1-28 Orange Flambe One-Coat Hide Egg...,"[0.014321798458695412, -0.024152863770723343, ...",0.491515,-0.066625
1327,P1327,5 gal. #MQ4-44 Green Dynasty Extra Durable Egg...,BEHR ULTRA SCUFF DEFENSE Stain-Blocking Paint ...,recommended,5 gal. #MQ4-44 Green Dynasty Extra Durable Egg...,"[0.04926867410540581, -0.019617563113570213, 0...",0.473392,-0.065307
1705,P1705,5 gal. #310D-4 Gold Buff Extra Durable Satin E...,BEHR ULTRA SCUFF DEFENSE Stain-Blocking Paint ...,recommended,5 gal. #310D-4 Gold Buff Extra Durable Satin E...,"[-0.002202090574428439, -0.013825895264744759,...",0.467007,-0.037536


In [25]:
px.scatter(data, x='pc1',y='pc2',color='product_status',hover_data="title")